# Create basic chat bot

This notebook creates the basic chatbot which can be used to get movie recommendation. It is not yet hooked up to the lambda function.

In [13]:
import boto3, os
import pandas as pd
import json
os.environ['AWS_DEFAULT_REGION']="us-east-1"
client = boto3.client('lex-models')

## Prepare slot values from dataset

In [14]:
movies = pd.read_csv('./movies.csv',  names=['ITEM_ID','title','genre'], engine= 'python', header = 0)
movies.head()

,ITEM_ID,title,genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
import re

def split_it(title):
    p = re.compile('(.*)(\(\d\d\d\d\))')
    if p.search(title):
        return p.search(title).group(1)
    else: 
        return title

titles =movies['title'].apply(split_it).unique()

In [16]:
slot_value_list = []

for val in titles[:3000].tolist():
    data = {}
    data['value'] = val
    slot_value_list.append(data)



## Create slot type for watched movies

In [17]:
#checksum = client.get_slot_type(name='watchedMovie', version='$LATEST')['checksum']
response = client.put_slot_type(
    name='watchedMovie',
    description='A movie which the user has already watched',
    valueSelectionStrategy='ORIGINAL_VALUE',
    enumerationValues=slot_value_list,
    createVersion=False
)

## Create movie recommendation intent

In [18]:

response = client.put_intent(
    name='RecommendMovieIntent',
    description='This intent allows to get movie recommendations',
    slots=[
        {
            'name': 'watchedMovie',
            'description': 'A movie that the user already watched',
            'slotConstraint': 'Required',
            'slotType': 'watchedMovie',
            'slotTypeVersion': '$LATEST',
            'valueElicitationPrompt': {
                'messages': [
                    {
                        'contentType': 'PlainText',
                        'content': 'To make a good recommendation. Tell me a movie which you really liked!'
                    },
                ],
                'maxAttempts': 3
            },
            'priority': 1
        },
    ],
    sampleUtterances=[
        'Can you recommend me a movie','Can you recommend me a movie that is similar to {watchedMovie}', 'Which movie should i watch', 'Which movie should i see', 'What is a great movie', "Tell me a great movie similar to {watchedMovie}"
    ],
    confirmationPrompt={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Do you want a movie similar to {watchedMovie}'

            }
        ],
        'maxAttempts': 1
    },
    rejectionStatement={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Okay, not recommending you a movie!'

            },
        ],
        'responseCard': 'string'
    },
    fulfillmentActivity={
        'type': 'ReturnIntent'        
    },
        createVersion=False
)

## Create the chatbot

In [20]:
response = client.put_bot(
    name='MovieRecommender',
    description='This bot provides movie recommendations based movies you have seen',
    intents=[
        {
            'intentName': 'RecommendMovieIntent',
            'intentVersion': '$LATEST'
        },
    ],
    clarificationPrompt={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Hi. If you want a movie recommendation, ask What is a great movie?'
                
            },
        ],
        'maxAttempts': 3
    },
    abortStatement={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Thanks for using the movie recommender bot. Goodbye!'
                
            },
        ]
    },
    idleSessionTTLInSeconds=120,
    voiceId='Joanna',
    processBehavior='BUILD',
    locale='en-US',
    childDirected=False,
    createVersion=False
)

## Delete resources

In [8]:
response = client.delete_bot(
    name='MovieRecommender'
)


In [10]:
response = client.delete_intent(
    name='RecommendMovieIntent'
)

In [ ]:
response = client.delete_slot_type(
    name='watchedMovie'
)